In [67]:
#lets upload train,test,submisison file in python environment

import pandas as pd
import numpy as np

In [68]:
random_state=42

In [69]:
#import train and test dataset
train = pd.read_csv('train_jqd04QH.csv')
test = pd.read_csv('test_GYi4Gz5.csv')


In [70]:
#import submission format
submission = pd.read_csv('sample_submission_sxfcbdx.csv')

In [71]:
#Lets look at strucutre of train,test,submission
print(train.shape)
print(test.shape)
print(submission.shape)

(18359, 14)
(15021, 13)
(15021, 2)


In [72]:
#log transformation for right skewed
train['training_hours'] = np.log(train['training_hours'])

In [73]:
#bins for city development_index
bins = [0, 0.75,0.85,0.99]
labels = ['l2','m','h5']
train['CITY_DVLPMNT_binned'] = pd.cut(train['city_development_index'], bins=bins, labels=labels)

In [74]:
#Lets explore train_part for modeling excercise.

#look at first few row of train_part

train.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,CITY_DVLPMNT_binned
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,4.663439,0,l2
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,4.234107,0,h5
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,1.386294,0,h5
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,3.258097,0,l2
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,4.477337,1,h5


In [75]:
# checking missing values in train
print(train.isnull().sum())

# percentage of missing values
print("\n\nPercentage of missing values:\n")
print(((train.isnull().sum())/train.shape[0])*100)

enrollee_id                  0
city                         0
city_development_index       0
gender                    4098
relevent_experience          0
enrolled_university        342
education_level            457
major_discipline          2838
experience                  59
company_size              4779
company_type              5039
last_new_job               367
training_hours               0
target                       0
CITY_DVLPMNT_binned          0
dtype: int64


Percentage of missing values:

enrollee_id                0.000000
city                       0.000000
city_development_index     0.000000
gender                    22.321477
relevent_experience        0.000000
enrolled_university        1.862847
education_level            2.489242
major_discipline          15.458358
experience                 0.321368
company_size              26.030830
company_type              27.447029
last_new_job               1.999020
training_hours             0.000000
target               

* Total 8 features have missing values in them.
* From these 4 features have around ~20% missing values.
* And 4 feature have less than 3 % missing values.
* For sure model is sensitive to missing value treatment exercise.

In [76]:
#small is for feature having less than 3% missing values.

small=['enrolled_university','last_new_job','education_level','experience'] #features having <2% missing values
for i in range(len(small)):  #Viewing unique values 
    print(small[i], 'values:')
    print(train[small[i]].unique(),'\n\n')
#checking levels in small    


enrolled_university values:
['no_enrollment' 'Full time course' 'Part time course' nan] 


last_new_job values:
['1' '2' '3' '>4' 'never' '4' nan] 


education_level values:
['Graduate' 'Masters' 'High School' 'Phd' nan 'Primary School'] 


experience values:
['3' '14' '6' '8' '>20' '4' '9' '15' '10' '1' '5' '16' '11' '12' '7' '2'
 '13' '<1' '19' '18' '17' nan '20'] 




In [77]:
''' One way to handle <2% MISSING is by mode imputation, lets perform it'''
train['enrolled_university'].fillna('no_enrollment',inplace=True)
train['last_new_job'].fillna('1',inplace=True)
train['education_level'].fillna('Graduate',inplace=True)
train['experience'].fillna('>20',inplace=True)



In [78]:
train.enrolled_university.value_counts()
#check if imputation was successful

no_enrollment       14001
Full time course     3187
Part time course     1171
Name: enrolled_university, dtype: int64

In [79]:
more=['gender','company_size','company_type','major_discipline'] 
#more is for features having missing values greater than 15%
for i in range(len(more)):
    more[i]
    train[more[i]].fillna('MISSING',axis=0,inplace=True)
#Filling missing values with mode is not a decent move as missing values frequency is
# too close to other level in same feature
# So created a new level 'MISSING' is being done.
    

### Hypothesis Excercise
(One by one)

1. lets create a new feature 'DISSATISFIED_WITH_SALARY' indicating if particular candidate is dissatisfied with his/her salary or not.

If an employed candidate taking lesser training_hours compared to other candidates with 
same qualification level. I am making it proxy with salary dis/satisfaction.

Assuming :
1. Candidate pyscologically compares with peers of same education level.
2. Candidate with similar education level earns same salary.
3. Spending behaviour depend on the earning. Since all have similar salary, candidate with 
   similar qualification must have joined same training_hours choice.
4. Higher the training_hours, due to course depth, highers will be the fees.




In [80]:
#creating new feature 'DISSATISFIED_WITH_SALARY

train.groupby(by='education_level',axis=0).mean() 

#check mean training_hours in each education_level

# Now before going into it we need information on if the candidate is employed or not.
# Since a person is dissatisfied with slary iff person is employed.



,enrollee_id,city_development_index,training_hours,target
education_level,,,,
Graduate,16860.399430,0.842175,3.773484,0.141903
High School,16770.493602,0.850658,3.788324,0.118602
Masters,16550.428340,0.852673,3.738649,0.118777
Phd,14803.514161,0.889773,3.764044,0.115468
Primary School,17045.578947,0.862963,3.692259,0.077399


2. Lets create a new feature indicating if the candidate is employed or not. As it is 
required by preveious 'DISSATISFIED_WITH_SALARY new feature creation exercise. For this I will use information from existing feature *enrolled_university*.

Assuming: 
1. If candidate is full time enrolled at university means that person is unemployed otherwise lets assume person is employed.

In [81]:
# create 'IS_EMPLOYED' feature indicating if the candidate is employed or not.

# Lets check all unique values of enrolled_university
train['enrolled_university'].unique().tolist()

train['IS_EMPLOYED'] = np.where(train.enrolled_university == 'Full time course', '0', '1')



As IS_EMPLOYED feature is created now I will create new feature 'DISSATISFIED_WITH_SALARY.


In [82]:
cond1=(train.IS_EMPLOYED == '1')&(train.education_level =='Graduate')&(train.training_hours <= 66)
cond2=(train.IS_EMPLOYED == '1')&(train.education_level =='High School')&(train.training_hours <= 66)
cond3=(train.IS_EMPLOYED == '1')&(train.education_level =='Masters')&(train.training_hours <= 63)
cond4=(train.IS_EMPLOYED == '1')&(train.education_level =='Phd')&(train.training_hours <= 68)
cond5=(train.IS_EMPLOYED == '1')&(train.education_level =='Primary School')&(train.training_hours <= 65)
train['DISSATISFIED_WITH_SALARY'] = np.where(cond1 | cond2 | cond3| cond4 | cond5,'1','0')

Lets create a feature showing if the person has really interest in DS:

A candidate having no prior experience of DS plus he/she is from /Arts/No Major background. Then he/she might be interested in DS domain. (scope for a new feature.)
    

In [83]:
cond6=(train.relevent_experience == 'No relevent experience')&(train.major_discipline == 'Arts')
cond7=(train.relevent_experience == 'No relevent experience')&(train.major_discipline == 'No Major')

train['PASSION_FOR_DS'] = np.where( cond6 | cond7,'1','0') 

What if person current job profile started demanding DS knowledge to tackle problem as DS is making its ways to different industry.:

Say a candidate has not any relevant experience and also having experience of 10+ years so it might be due to fact that his/her current job started demanding DS knowledge.(So a new feature would be constructed from combination of relevant_experience and experience i.e, no relevant experience with ~10+ yrs total experience)

In [84]:
#lets convert >20 to 21 and <1 to 0 for simplification
train['experience'] = train['experience'].str.replace('<1','0')
train['experience'] = train['experience'].str.replace('>20','21')

In [85]:
#lets create JOB_START_DEMANDING_DS_SKILLS
cond8 = (train.relevent_experience == 'No relevent experience')&(train.experience.astype('int64') > 4)

train['JOB_START_DEMANDING_DS_SKILLS'] = np.where( cond8,'1','0') 

If the candidate having relevant experience it is majorly possibly due to three big reasons:
1. Candidate has studies on his own @ college.
2. Has done a training prior to this training. So might be brushing up his knowledge with small training hours.
3. Candidate has studied it under his/her curriculam.

	Therefore, we can create a new feature: Candidate with small training hours plus has 		relevant experience are more likely to be repeating ones (say). On other side candidate with 	no relvant experience are definetely attending for first time.
	


In [86]:
#REPEATERS
cond9 = (train.training_hours < 16)&(train.relevent_experience =='Has relevent experience')
train['REPEATERS'] = np.where(cond9,'1','0')

In [87]:
#for XGBOOST only 
train['company_size'] = train['company_size'].str.replace('<10','LESS10')
train['last_new_job'] = train['last_new_job'].str.replace('>4','LARGE4')

In [88]:

category_cols = ['gender','relevent_experience','enrolled_university','education_level','major_discipline','experience','company_size','company_type','last_new_job']


for header in category_cols:
    train[header] = train[header].astype('category').cat.codes




In [89]:
'''Developing model'''
X= train.drop(['target','enrollee_id'],axis=1)
y= train['target']

X=pd.get_dummies(X)

#lets further train_part into two parts model tuning exercise.
from sklearn.cross_validation import train_test_split
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=7,stratify=y)

In [90]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier


In [91]:
switch = XGBClassifier(learning_rate=0.09,n_estimators=125,max_depth=4,min_child_weight=4,colsample_bytree=0.5,reg_alpha=0.000001 )

In [92]:
from sklearn.model_selection import cross_val_score

In [93]:
from sklearn.metrics import roc_auc_score


In [54]:
#cross validation score
Accuracies = cross_val_score(switch,X_trainset,y_trainset,cv=8,scoring= 'roc_auc')

In [55]:
Accuracies 

array([0.64473724, 0.62469061, 0.65545566, 0.66701632, 0.6797833 ,
       0.66481687, 0.65606474, 0.67711181])

In [94]:
np.mean(Accuracies)

In [95]:
'''Developing model on full train'''
switch.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.09, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=125,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1e-06, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### Lets predict for test dataset  


In [96]:
#log transformation for right skewed
test['training_hours'] = np.log(test['training_hours'])

In [97]:
#bins for city development_index
bins = [0, 0.75,0.85,0.99]
labels = ['l2','m','h5']
test['CITY_DVLPMNT_binned'] = pd.cut(test['city_development_index'], bins=bins, labels=labels)

In [98]:
# checking missing values in train
print(test.isnull().sum())

# percentage of missing values
print("\n\nPercentage of missing values:\n")
print(((test.isnull().sum())/test.shape[0])*100)

enrollee_id                  0
city                         0
city_development_index       0
gender                    3388
relevent_experience          0
enrolled_university        279
education_level            395
major_discipline          2393
experience                  44
company_size              4051
company_type              4330
last_new_job               304
training_hours               0
CITY_DVLPMNT_binned          0
dtype: int64


Percentage of missing values:

enrollee_id                0.000000
city                       0.000000
city_development_index     0.000000
gender                    22.555090
relevent_experience        0.000000
enrolled_university        1.857400
education_level            2.629652
major_discipline          15.931030
experience                 0.292923
company_size              26.968910
company_type              28.826310
last_new_job               2.023833
training_hours             0.000000
CITY_DVLPMNT_binned        0.000000
dtype: float64


In [99]:
small=['enrolled_university','last_new_job','education_level','experience'] #features having <2% missing values
for i in range(len(small)):  #Viewing unique values 
    print(small[i], 'values:')
    print(test[small[i]].unique(),'\n\n')
    


enrolled_university values:
['Full time course' 'no_enrollment' 'Part time course' nan] 


last_new_job values:
[nan '1' '2' '>4' 'never' '4' '3'] 


education_level values:
['Graduate' 'Phd' 'High School' 'Masters' nan 'Primary School'] 


experience values:
['<1' '>20' '3' '6' '7' '9' '18' '2' '10' '14' '16' '5' '19' '17' '4' '8'
 '12' '13' '20' '15' '1' '11' nan] 




In [100]:
''' One way to handle <2% MISSING is by mode imputation, lets perform it'''
test['enrolled_university'].fillna('no_enrollment',inplace=True)
test['last_new_job'].fillna('1',inplace=True)
test['education_level'].fillna('Graduate',inplace=True)
test['experience'].fillna('>20',inplace=True)



In [101]:
test.enrolled_university.value_counts()

no_enrollment       11507
Full time course     2565
Part time course      949
Name: enrolled_university, dtype: int64

In [102]:
more=['gender','company_size','company_type','major_discipline'] 
#features having missing values
for i in range(len(more)):
    more[i]
    test[more[i]].fillna('MISSING',axis=0,inplace=True)

    

In [103]:
test['IS_EMPLOYED'] = np.where(test.enrolled_university == 'Full time course', '0', '1')



In [104]:
cond1=(test.IS_EMPLOYED == '1')&(test.education_level =='Graduate')&(test.training_hours <= 66)
cond2=(test.IS_EMPLOYED == '1')&(test.education_level =='High School')&(test.training_hours <= 66)
cond3=(test.IS_EMPLOYED == '1')&(test.education_level =='Masters')&(test.training_hours <= 63)
cond4=(test.IS_EMPLOYED == '1')&(test.education_level =='Phd')&(test.training_hours <= 68)
cond5=(test.IS_EMPLOYED == '1')&(test.education_level =='Primary School')&(test.training_hours <= 65)
test['DISSATISFIED_WITH_SALARY'] = np.where(cond1 | cond2 | cond3| cond4 | cond5,'1','0')

In [105]:
cond6=(test.relevent_experience == 'No relevent experience')&(test.major_discipline == 'Arts')
cond7=(test.relevent_experience == 'No relevent experience')&(test.major_discipline == 'No Major')

test['PASSION_FOR_DS'] = np.where( cond6 | cond7,'1','0') 

In [106]:
#lets convert >20 to 21 and <1 to 0 for simplification
test['experience'] = test['experience'].str.replace('<1','0')
test['experience'] = test['experience'].str.replace('>20','21')

In [107]:
#lets create JOB_START_DEMANDING_DS_SKILLS
cond8 = (test.relevent_experience == 'No relevent experience')&(test.experience.astype('int64') > 4)

test['JOB_START_DEMANDING_DS_SKILLS'] = np.where( cond8,'1','0') 

In [108]:
cond9 = (test.training_hours < 16)&(test.relevent_experience =='Has relevent experience')
test['REPEATERS'] = np.where(cond9,'1','0')

In [109]:
#for XGBOOST only
test['company_size'] = test['company_size'].str.replace('<10','LESS10')
test['last_new_job'] = test['last_new_job'].str.replace('>4','LARGE4')

In [110]:

category_cols = ['gender','relevent_experience','enrolled_university','education_level','major_discipline','experience','company_size','company_type','last_new_job']

for header in category_cols:
    test[header] = test[header].astype('category').cat.codes

In [111]:
'''predicting test target'''
X_test= test.drop('enrollee_id',axis=1)


X_test=pd.get_dummies(X_test)


In [112]:
test_target=switch.predict_proba(X_test)[:,1]

In [113]:
submission['target'] = test_target

In [114]:
submission.head()

,enrollee_id,target
0,16548,0.510850
1,12036,0.066723
2,11061,0.362608
3,5032,0.058193
4,17599,0.095439


In [115]:
submission.to_csv('afterSubmission.csv',index=False)